In [14]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

In [18]:
# loading and preprocessing (data normailzation and augmentation) the data using transforms 
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Randomly crop and resize
        transforms.RandomHorizontalFlip(),  # Randomly flip horizontally
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
}


data_dir = 'data'
image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'test']}


In [19]:
#loading pre-trained resnet model

resnet = models.resnet18(pretrained=True)

In [20]:
len(image_datasets['train'].classes)

1

In [21]:
# Freeze all layers except the last classification layer to avoid forgetting the previous training on Imagenet

for param in resnet.parameters():
    param.requires_grad = False

# Modify the last classification layer to 1 class for eyeglasses
num_classes = len(image_datasets['train'].classes)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)


In [24]:
# defining loss function and optimizer with weight decay  for fine-tuning the resnet model 

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD([
    {'params': resnet.fc.parameters(), 'lr': 0.001},
    {'params': [param for name, param in resnet.named_parameters() if 'fc' not in name], 'weight_decay': 0.0001, 'lr': 0.0001},
], momentum=0.9)



In [25]:
# fine tuning the model 
for epoch in range(5):  # Change the number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(dataloaders['train'], 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished fine-tuning')


Finished fine-tuning


In [26]:
# testing the model

correct = 0
total = 0

with torch.no_grad():
    for data in dataloaders['test']:
        images, labels = data
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on test images: {accuracy:.2%}')


Accuracy on test images: 100.00%
